In [24]:
import pandas as pd

In [38]:
df = pd.read_csv('market_basket.csv',sep=";")

C:\Users\rodri\AppData\Local\Temp\ipykernel_22040\1469006059.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('market_basket.csv',sep=";")


In [39]:
df.Itemname = df.Itemname.astype('str')

In [40]:
items_by_basket = [list(group['Itemname']) for _, group in df.groupby('BillNo')]

In [50]:
items_by_basket = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [57]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules

# convert list of transactions to a DataFrame
te = TransactionEncoder()
te_ary = te.fit_transform(items_by_basket)
df = pd.DataFrame(te_ary, columns=te.columns_)

# apply Apriori to find frequent itemsets
frequent_itemsets = apriori(df, use_colnames=True)


In [58]:
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Eggs, Onion)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Kidney Beans, Onion)"
9,0.6,"(Kidney Beans, Yogurt)"


In [63]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [64]:
# filter the rules to show only the ones with "Eggs" in the antecedent
egg_rules = rules[rules['antecedents'].apply(lambda x: 'Eggs' in x)]

# sort the rules by confidence in descending order
egg_rules = egg_rules.sort_values(by='confidence', ascending=False)

In [65]:
egg_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf
9,"(Eggs, Onion)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
2,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
7,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
10,(Eggs),"(Kidney Beans, Onion)",0.8,0.6,0.6,0.75,1.25,0.12,1.6


The confidence measures the conditional probability of the consequent (right-hand side) given the antecedent (left-hand side) of a rule.

In [66]:
# display the recommended items to buy
recommended_items = list(egg_rules.iloc[0]['consequents'])
print(f"Based on the purchase of 'Eggs', the most recommended item to buy is: {recommended_items[0]}")

Based on the purchase of 'Eggs', the most recommended item to buy is: Kidney Beans
